In [1]:
# %pip install -r C:\Users\jank9\Amort\requirements.txt
# %pip install pickleshare

In [2]:
from itertools import accumulate
from dash import Dash, dcc, callback, Output, Input, State, ALL, callback_context, html, no_update
from dash.exceptions import PreventUpdate
from dash_iconify import DashIconify # type: ignore
import plotly.express as px
import dash_mantine_components as dmc # type: ignore
import time
import numpy as np

In [3]:
%cd C:\Users\jank9\Amort
from app.Dashboard.assets.ids import GRAPH, LOAN
from app.Loan import df_schema, calculator, default_kwargs, example_for_subsidy_arr

C:\Users\jank9\Amort


In [4]:
memory= calculator(**default_kwargs, thousand_sep= False)
default_kwargs_subsidy = default_kwargs.copy()
default_kwargs_subsidy['subsidy_arr'] = example_for_subsidy_arr
memory_subsidy = calculator(**default_kwargs_subsidy, thousand_sep= False)


target= [
    df_schema.level_2.PAYMENT, 
    df_schema.level_2.PRINCIPAL, 
    df_schema.level_2.INTEREST
]

# memory= memory_subsidy
pie_data = {}
payment= {}
for key, value in {k: round(v) for k, v in zip(memory['columns'], memory['data'][-1]) if k[-1] in target}.items():
    if key[-2] not in pie_data.keys():
        pie_data.update({key[-2]: {'labels': [], 'values': [], 'names': []}}) # initialize the dictionary
    if key[-1] == df_schema.level_2.PAYMENT:
        if key[-2] not in payment.keys():
            payment.update({key[-2]: {'labels': [], 'values': [], 'names': []}}) # initialize the dictionary
        payment[key[-2]]['labels'].append(key[-1])
        payment[key[-2]]['values'].append(value)
        payment[key[-2]]['names'].append((key[0] if key[0] != key[-2] else None))
    else:
        pie_data[key[-2]]['labels'].append(key[-1])
        pie_data[key[-2]]['values'].append(value)
        pie_data[key[-2]]['names'].append((key[0] if key[0] != key[-2] else None))

payment
# [v for sub in [pie_data[k]['names'] for k in pie_data.keys()] for v in sub]
# pie_data


{'本息攤還法': {'labels': ['償還總額'], 'values': [10558837], 'names': [None]},
 '本金攤還法': {'labels': ['償還總額'], 'values': [10334467], 'names': [None]}}

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import numpy as np

fig = None
fig_main= None
fig_sub= None


for col, method in enumerate(pie_data.keys()):
    if len(pie_data[method]['names']) == 2:
        if not fig:
            fig = make_subplots(rows=1, cols=len(pie_data.keys()), specs=[[{'type':'domain'}, {'type':'domain'}]])
        fig.add_trace(
            go.Pie(
                labels= pie_data[method]['labels'],
                values= pie_data[method]['values'],
                title= f"""<b>{method}</b><br>{round(payment[method]['values'][0] // len(memory['data'])): ,}/月""",
                name= "",
                titlefont= dict(size= 20, color= 'dark grey'),
                marker= dict(colors= px.colors.qualitative.D3),
                textfont= dict(size= 15),
                textinfo= 'percent',
                textposition= 'inside',
                hoverinfo= 'label+value',
                hovertemplate= '%{label}: %{value:,.0f}',
                showlegend= False,
                hole= .7,

            ),
                1, 
                col + 1,
        )
        fig.update_traces(
            hole= .7,
            hoverinfo="label+percent+name",
        )
        fig.update_layout(
            title_text="<b>Total Payment</b>", 
            title_font_size= 20,
            title_x= 0.5,
        )
            
    else:
        for name in pie_data[method]['names']:
            if name == df_schema.level_0.ORIGINAL:
                if not fig_main:
                    fig_main = make_subplots(rows=1, cols=len(pie_data.keys()), specs=[[{'type':'domain'}, {'type':'domain'}]])
                fig_main.add_trace(
                    go.Pie(
                        labels= pie_data[method]['labels'],
                        values= pie_data[method]['values'],
                        title= f"""<b>{method}</b><br>{round([v for c, v in zip(zip(payment[method]['labels'], payment[method]['names']), payment[method]['values'])  if c[0] == df_schema.level_2.PAYMENT and c[1] == name][0] // len(memory['data'])): ,}/月""",
                        name= "",
                        titlefont= dict(size= 20, color= 'dark grey'),
                        marker= dict(colors= px.colors.qualitative.D3),
                        textfont= dict(size= 15),
                        textinfo= 'percent',
                        textposition= 'inside',
                        hoverinfo= 'label+value',
                        hovertemplate= '%{label}: %{value:,.0f}',
                        showlegend= False,
                    ),
                        1, 
                        col + 1,
                )
                fig_main.update_traces(
                    hole= .7,
                    hoverinfo="label+percent+name",
                )
                fig_main.update_layout(
                    title_text="<b>Original Loan</b>", 
                    title_font_size= 20,
                    title_x= 0.5,
                )
            elif name == df_schema.level_0.SUBSIDY:
                if not fig_sub:
                    fig_sub = make_subplots(rows=1, cols=len(pie_data.keys()), specs=[[{'type':'domain'}, {'type':'domain'}]])
                fig_sub.add_trace(
                    go.Pie(
                        labels= pie_data[method]['labels'],
                        values= pie_data[method]['values'],
                        title= f"""<b>{method}</b><br>{round([v for c, v in zip(zip(payment[method]['labels'], payment[method]['names']), payment[method]['values'])  if c[0] == df_schema.level_2.PAYMENT and c[1] == name][0] // len(memory['data'])): ,}/月""",
                        name= "",
                        titlefont= dict(size= 20, color= 'dark grey'),
                        marker= dict(colors= px.colors.qualitative.D3),
                        textfont= dict(size= 15),
                        textinfo= 'percent',
                        textposition= 'inside',
                        hoverinfo= 'label+value',
                        hovertemplate= '%{label}: %{value:,.0f}',
                        showlegend= False,
                    ),
                        1, 
                        col + 1,
                )
                fig_sub.update_traces(
                    hole= .7,
                    hoverinfo="label+percent+name",
                )
                fig_sub.update_layout(
                    title_text="<b>Subsidy Loan</b>", 
                    title_font_size= 20,
                    title_x= 0.5,
                )
        


import dash_bootstrap_components as dbc
app= Dash(
    __name__, 
    suppress_callback_exceptions=True, 
    external_stylesheets=[
        dbc.themes.LUMEN, 
        dbc.icons.BOOTSTRAP,
        "app/Dashboard/assets/style.css",
        ]
    )
app.layout= html.Div(
    children= [
        dcc.Graph(
            figure= fig,
            style= {'width': '100%', 'height': '100%'},
            config= {
                'displayModeBar': False,
                'scrollZoom': False,
                'editable': False,
                'displaylogo': False,
            },
        ) for fig in [f for f in[fig, fig_main, fig_sub] if f]
    ],  
    style= {
            'display': 'flex',
            'width': '100%',
            'height': '40%',
        },
)
app.run_server(
    debug=True, 
    port= 8050, 
    dev_tools_hot_reload= True, 
    dev_tools_hot_reload_interval= 1000, 
    threaded= True
)

In [99]:
def graph():
    """Build the graph"""
    layout = dmc.Container(
        [
            dmc.Group(
                [
                    html.Div(
                        dmc.LoadingOverlay(
                            id= 'loading-overlay-for-general',
                            children= [],
                            loaderProps={
                                "variant": "oval",
                                 "color": "blue", 
                                 "size": "lg",
                                 'is_loading': True,
                            },
                            transitionDuration= 0.5,
                        ),
                        style={
                            'border-radius': '5px',
                            'border': '1px solid #ccc',
                            'border-radius': '5px',
                            'height': 150,
                            'background-color': 'white',
                        },
                    ),
                    html.Div(
                        dmc.LoadingOverlay(
                                id= 'loading-overlay-for-details',
                                children= [],
                            loaderProps={"variant": "oval",
                                 "color": "blue", 
                                 "size": "lg",
                                 'is_loading': True,
                            },
                            style= {
                                'margin-top': '10px',
                            },
                            transitionDuration= 0.5,
                            

                        ),
                        style={
                            'border-radius': '5px',
                            'border': '1px solid #ccc',
                            'height': 150,
                            'background-color': 'white',
                        },
                    ),
                ],
                grow=True,
                style= {
                    'margin-top': '10px',
                }    
            ),
            dmc.Space(h=20),
            dmc.Group(
                [
                    dmc.Menu(
                        [
                            dmc.MenuTarget(
                                dmc.Button(
                                    id='menu-target',
                                    children=df_schema.level_2.PAYMENT,
                                    variant="gradient",
                                    gradient={"from": "indigo", "to": "cyan"},
                                    style={
                                        'width': '110px',
                                    },
                                    loading={'loading_type': 'overlay'},
                                    loaderPosition='center',
                                    loaderProps={"variant": "dots",
                                                 "color": "white", "size": "sm"},
                                ),
                            ),
                            dmc.MenuDropdown(
                                [],
                                id='menu-dropdown',
                            ),
                        ],
                        id=GRAPH.DROPDOWN.MENU,
                    ),
                    dmc.SegmentedControl(
                        id=GRAPH.ACCUMULATION,
                        value="regular",
                        data=[
                            {"value": "regular", "label": "Regular"},
                            {"value": "cumulative", "label": "Cumulative"},
                        ],
                        size='sm',
                        style={
                            'background-color': "rgba(0, 62, 143, 0.29)",
                            'box-shadow': '0 0 5px #ccc',
                        },
                    ),
                ],
                spacing=10,
                position='left',
                style={
                    'width': '100%',
                }
            ),
            dmc.Space(h=20),
            html.Div(
                dmc.LoadingOverlay(
                    id= 'loading-overlay-for-main-graph',
                    loaderProps={"variant": "oval",
                                 "color": "blue", 
                                 "size": "lg",
                                 'is_loading': True,
                                 },
                    transitionDuration= 0.5,
                ),
                style={
                    'height': 600,
                },
            ),
        ],
        fluid=True,
        style={
            'width': '100%',
            'height': 'auto',
            'margin-top': '5px',
        }
    )

#1 options for dropdown menu of the main graph
    @callback(
        Output('menu-dropdown', 'children'),
        Output('menu-dropdown', 'style'),
        Input(GRAPH.ACCUMULATION, 'value'),
        Input('menu-target', 'children'),
        Input(LOAN.RESULT.DATAFRAME, 'data'),
    )
    def update_toggle_items(
        accum,
        label,
        data,
    ):
        data = data['data']
        level_0 = {c[0] for c in data['columns']}
        if df_schema.level_0.TOTAL in level_0:
            return [], {'display': 'none'}
        else:
            columns_for_original_mortgate = [
                df_schema.level_2.PAYMENT,
                df_schema.level_2.PRINCIPAL,
                df_schema.level_2.INTEREST,
                df_schema.level_2.RESIDUAL,
            ]
            if accum and accum == 'cumulative':
                columns_for_original_mortgate = [
                    v for v in columns_for_original_mortgate if v != df_schema.level_2.RESIDUAL]
            dropdown = [
                dmc.MenuItem(
                    column,
                    id={
                        'index': column,
                        'type': GRAPH.DROPDOWN.ITEM
                    },
                ) for column in columns_for_original_mortgate if column != label
            ]  # avoid duplicate label
            return dropdown, {'display': 'block'}

#2 update the main graph
    @callback(
        Output('loading-overlay-for-main-graph', 'children'),
        Output('menu-target', 'children'),
        Output('menu-target', 'rightIcon'),
        Output('menu-target', 'loading'),
        Output('menu-target', 'gradient'),
        Input(GRAPH.ACCUMULATION, 'value'),
        Input(LOAN.RESULT.DATAFRAME, 'data'),
        Input({'index': ALL, 'type': GRAPH.DROPDOWN.ITEM}, 'n_clicks'),
        State('menu-target', 'children'),
    )
    def update_graph(
        accum,
        data,
        _,
        label,
    ):
        ctx = callback_context
        data = data['data']
        
        if isinstance(ctx.triggered_id, dict):
            chosen_figure = ctx.triggered_id['index']
        else:
            # type: ignore
            if df_schema.level_0.TOTAL not in [col[0] for col in data['columns']]:
                if accum == 'cumulative':
                    if label == df_schema.level_2.RESIDUAL:
                        chosen_figure = df_schema.level_2.PAYMENT
                    else:
                        chosen_figure = label
                else:
                    chosen_figure = (
                        label if label != df_schema.level_0.TOTAL else df_schema.level_2.PAYMENT)
            else:
                chosen_figure = df_schema.level_0.TOTAL
        
        ns, cols = zip(*[(n, col) for n, col in enumerate(data['columns']) if chosen_figure in col])
        x_axis_value = data['index'][1:-1]
        
        # construct the data frame for the graph
        data_frame_for_loan_timeSeries= {
            'Time': [],
            'Amount': [],
            'labels': [],
            'methods': [],
        }

        for n, col in zip(ns, cols): # n: index of the column, col: column name
            dff = [data[n] for data in data['data'][1:-1]]
            method = (" + ".join([co for co in col if co != chosen_figure]) if len(col) > 2 else col[0])
            if accum == 'cumulative':
                dff = [*accumulate(dff)]
            dff = [*map(lambda x: f"{round(x):,}", dff)]
            
            data_frame_for_loan_timeSeries['Time'].extend(x_axis_value)
            data_frame_for_loan_timeSeries['Amount'].extend(dff)
            data_frame_for_loan_timeSeries['labels'].extend([method] * len(x_axis_value))
            data_frame_for_loan_timeSeries['methods'].extend([method] * len(x_axis_value))
            
        fig = px.line(
            data_frame_for_loan_timeSeries, 
            x= "Time", 
            y="Amount", 
            color="methods", 
            title='<b>Life of Loan</b>', 
            # width=1000, 
            height=600,
            log_y= True,
            line_shape="spline",
            render_mode="svg",
            hover_name="methods",
            template="plotly_white",
            color_discrete_map= {
                    df_schema.level_1.ETP: '#0C82DF',
                    df_schema.level_1.EPP: '#F7DC6F',
            },   
        )
        fig.update_layout(showlegend= False)

        return dcc.Graph(
                        figure= fig,
                        id=GRAPH.LINE,
                        hoverData= {'points': [{'x': '0', 'hovertext': 'ETP'}]},
                        config={
                            'displayModeBar': False,
                            'responsive': True,
                            'doubleClick': 'reset+autosize',
                            'toImageButtonOptions': {
                                'format': 'svg',
                                'filename': 'custom_image',
                                'height': 600,
                                'width': 1000,
                                'scale': 1,
                            },
                        },
                        style={
                            'width': '100%',
                            'height': 600,
                            'margin-top': '5px',
                            'border-radius': '5px',
                            'border': '1px solid #ccc',
                        },
                    ), chosen_figure, (DashIconify(icon="raphael:arrowdown") if chosen_figure != df_schema.level_0.TOTAL else None), False, ({"from": "teal", "to": "blue", "deg": 60} if chosen_figure == df_schema.level_0.TOTAL else {"from": "indigo", "to": "cyan"})

#3 callback for updating the information from hoverData
    @callback(
        Output('loading-overlay-for-general', 'children'),
        Input(LOAN.RESULT.DATAFRAME, 'data'),
    )
    def update_general_info(
        memory
        ):
        bar_layout= dict(
            showlegend= False,
            margin= dict(l=20, r=10, t=35, b=50),
            paper_bgcolor= 'rgba(0, 0, 0, 0)',
            plot_bgcolor= 'rgba(0, 0, 0, 0)',
        )
        if memory:
            bars_data = {
                'columns': list({col[0] for col in memory['data']['columns']}),
                'data': []
            }
            if bars_data['columns'][0]:
                bars_data['data']= [
                    round([da for col, da in zip(memory['data']['columns'], memory['data']['data'][-1]) if col[0]== chosen_col and col[1]== df_schema.level_2.PAYMENT][0]/ len(memory['data']['index'][1:-1]))
                        for chosen_col in bars_data['columns']
                ]
                figure= px.bar(
                    bars_data,
                    x= 'data',
                    y= 'columns',
                    text= 'data',
                    text_auto= True,                
                    color= 'columns',
                    height= 180,
                    width= 400,
                    title= '<b>Average Payment</b>',
                    color_discrete_map= {
                        df_schema.level_1.ETP: '#0C82DF',
                        df_schema.level_1.EPP: '#F7DC6F',
                    },
                    hover_data= {
                        'columns': False,
                        'data': True,
                    },
                    hover_name= 'columns',
                )
                figure.update_layout(bar_layout)
                figure.update_yaxes(
                    visible= False,
                    showticklabels= False,
                )
                figure.update_xaxes(
                    range= [0, max(bars_data['data']) * 1.1],
                    visible= False,
                    showticklabels= False,
                )
                figure.update_traces(
                    texttemplate='%{text:,}',

                )
            else:
                figure= px.bar()
                figure.update_layout(bar_layout)
                figure.update_yaxes(
                    visible= False,
                    showticklabels= False,
                )
                figure.update_xaxes(
                    visible= False,
                    showticklabels= False,
                )
            return dcc.Graph(
                    figure= figure,
                    animate= True,
                    style={
                        'background-color': 'rgba(0, 0, 0, 0)',
                        'margin-top': '10px',
                        'width': '100%',
                    },
                    config={
                        'displayModeBar': False,
                        'doubleClick': 'reset+autosize',
                        'autosizable': True,
                        'editable': False,
                        'scrollZoom': False,
                        'staticPlot': False,
                    }
            )
        else:
            raise PreventUpdate

#4 callback for updating the detail information from hoverData
    @callback(
        Output('loading-overlay-for-details', 'children'),
        Input(GRAPH.LINE, 'hoverData'),
        State(LOAN.RESULT.DATAFRAME, 'data'),
    )
    def hover_data(
        hover_data,
        memory
        ):
        bar_layout= dict(
            showlegend= False,
            margin= dict(l=20, r=10, t=35, b=40),
            paper_bgcolor= 'rgba(0, 0, 0, 0)',
            plot_bgcolor= 'rgba(0, 0, 0, 0)',
        )
        if memory:
            timepoint= hover_data['points'][0]['x']
            
            if timepoint != '0':
                chosen_figure= hover_data['points'][0]['hovertext']
                bar_data = {
                    'names': chosen_figure,
                    'items': ['principal', 'interest', 'residual'],
                    'data': [],
                    'index': 0,
                }
                principal = round(
                            np.sum(
                                    [
                                        [da for col, da in zip(memory['data']['columns'], data) 
                                         if col[0]== chosen_figure and col[1]== df_schema.level_2.PRINCIPAL] 
                                         for data in memory['data']['data'][1:memory['data']['index'].index(timepoint)+1]
                                    ]
                                )
                            )
                interest = round(
                            np.sum(
                                    [
                                        [da for col, da in zip(memory['data']['columns'], data) 
                                         if col[0]== chosen_figure and col[1]== df_schema.level_2.INTEREST] 
                                         for data in memory['data']['data'][1:memory['data']['index'].index(timepoint)+1]
                                    ]
                                )
                            )
                residual = round([data for col, data in zip(memory['data']['columns'], memory['data']['data'][memory['data']['index'].index(timepoint)]) if col[0]== chosen_figure and col[1]== df_schema.level_2.RESIDUAL][0])
                bar_data['data']= [principal, interest, residual]

                fig= px.bar(
                    bar_data,
                    x= 'data',
                    y= 'index',
                    text= 'data',
                    text_auto= True,                
                    color= 'items',
                    height= 180,
                    # width= 480,
                    orientation= 'h',
                    title= '<b>Payment Breakdown</b>',
                    color_discrete_map= {
                        'principal': '#FF7F50',
                        'interest': '#F7DC6F',
                        'residual': '#6495ED',
                    },
                    hover_data= {
                        'items': False,
                        'index': False,
                        'data': True,
                    },
                    hover_name= 'items',
                )
                fig.update_layout(bar_layout)
                fig.update_yaxes(
                    visible= False,
                    showticklabels= False,
                )
                fig.update_xaxes(
                    # dynamcally range the x axis
                    range= [0, sum(bar_data['data']) * 1.1],
                    visible= False,
                    showticklabels= False,
                )
                fig.update_traces(
                    texttemplate='%{text:,}',
                    width= 0.5,
                )
                return dcc.Graph(
                        figure= fig,
                        animate= True,
                        style={
                            'background-color': 'rgba(0, 0, 0, 0)',
                            'margin-top': '10px',
                            'width': '100%', 
                        },
                        config={
                            'displayModeBar': False,
                            'doubleClick': 'reset+autosize',
                            'autosizable': True,
                            'editable': False,
                            'scrollZoom': False,
                            'staticPlot': False,
                        }
                )
            else:
                return no_update
        else:
            raise PreventUpdate
       
    return layout

In [ ]:
from app.Dashboard.pages.components.Controls.panels import panel
from app.Dashboard.assets import ids, specs
import dash_bootstrap_components as dbc

In [12]:
from app.Dashboard.pages.components.Controls.panels import panel
from app.Dashboard.assets import ids, specs
import dash_bootstrap_components as dbc

if __name__ == '__main__':
    app = Dash(
        __name__,
        assets_folder= 'app/Dashboard/assets/',
        suppress_callback_exceptions=True,
        external_stylesheets=[
        "https://fonts.googleapis.com/css2?family=Inter:wght@100;200;300;400;500;900&display=swap",
        dbc.themes.LUMEN,  # Dash Themes CSS
        dbc.icons.BOOTSTRAP,
        "https://use.fontawesome.com/releases/v6.2.1/css/all.css",  # Font Awesome Icons CSS
    ],
    )

    panel_tabs= dmc.Tabs(
            [
                dmc.TabsList(
                    [
                        dmc.Tab(
                            children= 'Main',
                            id= 'main-options',
                            #    icon= DashIconify(icon="bi:graph-up"),
                            value= 'Main',
                            style= {
                                'font-weight': 'bold',
                              },
                            ),
                        dmc.Tab(
                            children= 'Advanced',
                            id= 'advanced-options',
                            # icon= DashIconify(icon="ph:table-light"),
                            value= 'Advanced',
                            icon=DashIconify(icon="tabler:settings"),
                            style= {
                                'font-weight': 'bold',
                              },
                        ),
                    ],
                    grow= True,
                ),
                dmc.TabsPanel(panel.front(), value= 'Main'),
                dmc.TabsPanel(
                    panel._advancedoptions(), 
                    value= 'Advanced',
                ),
            ],
            value= 'Main',
            id="options-tabs",
            activateTabWithKeyboard= True,
            className= 'custom-scrollbar',
            variant= 'outline',
            style= {
                'width': '100%',
                'height': '100%',
                'scrollbar-color': '#0C82DF #E2E2E2',
                'color': '#333',
                'background-color': 'white',
            },

        )

    app.layout = dmc.MantineProvider(
        id= ids.APP.LOADING,
        theme= {
            "fontFamily": "'Inter', sans-serif",
            "primaryColor": "indigo",
            "components": {
                "Button": {"styles": {"root": {"fontWeight": 400}}},
                "Alert": {"styles": {"title": {"fontWeight": 500}}},
                "AvatarGroup": {"styles": {"truncated": {"fontWeight": 500}}},
            },
        },
        children=[
            panel.register(),
            # NAVBAR,
            dmc.Container(
                [
                    
                    html.Div(
                        children= [
                            panel_tabs,
                        ],
                        className= 'custom-scrollbar',
                        style= {
                            'width': '25dvw',
                            'height': '98vh',
                            'margin-top': 10,
                            'margin-left': 15,
                            'overflow-y': 'scroll',
                            'scrollbar-color': '#0C82DF #E2E2E2',
                            'border': '1px solid #ccc',
                            'border-radius': '5px',
                            'font-size': '20px',
                            'font-weight': 'bold',
                            'padding': '20px',
                            'color': '#333',
                            'background-color': 'white',
                        },
                    ),
                    html.Div(
                        [
                            graph(),
                        ],
                        className= 'custom-scrollbar',
                        style= {
                            'width': 'calc(100dvw - 365px)',
                            'height': '98vh',
                            'overflow-y': 'scroll',
                            'scrollbar-color': '#0C82DF #E2E2E2',
                            'margin-left': 5,
                        }
                    ),
                    
                ],
                fluid= True,
                style= {
                    'background-color': 'rgba(246,248,250,255)',
                    'width': '100vw',
                    'height': '98vh',
                    'margin': 0,
                    'padding': 0,
                    'display': 'flex',
                }
            )
        ],
        withGlobalStyles= True,
    )
    app.run_server(
    mode= 'external', # ['external', 'inline', 'jupyterlab', 'flask', 'dash', 'dash_jupyter'],
    dev_tools_ui=True,
    dev_tools_hot_reload= True,
    threaded=True,
    port= 8050,
    dev_tools_hot_reload_interval= 1000,
    )